<a href="https://colab.research.google.com/github/hasanzeynal/Tensorflow-Developer-Certificate/blob/main/01_Neural_Network_Regression_in_TensorFlow/neural_network_regression_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Regression with Neural Network in TensorFlow

In [1]:
#import tensorflow
import tensorflow as tf
print(tf.__version__)

2.12.0


# Basic Neural Network Model for Regression Problem
Then, After building our basic NN model, we are gonna try:
* improving our model
* evaluating
* setting
* saving
* loading
* processing with feature scalling(normalising, fitting model on a scaled data)

## Building Basic Model in TensorFlow

## Creating Data, So we can model it

In [81]:
X = tf.constant([-1,1,3,5,7,9,11,13,15,17,19,21,23])
y = tf.constant([9,11,13,15,17,19,21,23,25,27,29,31,33])  #the difference between X and y is 10

In [82]:
#looking for the input and output shape
X.shape,y.shape  #just we are gonna need comma in the tensoor shape after 14 so shape is (14,)

(TensorShape([13]), TensorShape([13]))

### Steps in Modelling in TensorFlow.
1. **Creating Model** - define imput, output, hidden layers of our Deep Learning Model.

2. **Compiling Model** - define the loss function(the function which tell us how wrong our model is), define optimizer(which tell us our model how improve) and evaluation metrics(we can use to interpret the performance of our model).

3. Fitting Model - letting the model try to find patterns between X and y(features and labels)

In [83]:
#change 1 rank tensoir to the 2 rank tensor
tf.expand_dims(X,-1)

<tf.Tensor: shape=(13, 1), dtype=int32, numpy=
array([[-1],
       [ 1],
       [ 3],
       [ 5],
       [ 7],
       [ 9],
       [11],
       [13],
       [15],
       [17],
       [19],
       [21],
       [23]], dtype=int32)>

In [84]:
#creating model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1)
])
#adding dense layer with the 1 neuron
#model.add(tf.keras.layers.Dense(1, input_shape = (14,)))
#model.add(tf.keras.layers.Dense(1))

#compiling the model
model.compile(loss= tf.keras.losses.mae,  #MAE stands for the Mean Absolute Error
              optimizer= tf.keras.optimizers.SGD(), #SGD stands for the Stochastic Gradient Error
              metrics= ['mae'])

#fitting the model
model.fit(tf.expand_dims(X,axis = -1), y, epochs= 5)  #axis= -1 makes 2 rank tensor,1 rank tensor is caused an error

Epoch 1/5
1/1 [==============================] - 0s 365ms/step - loss: 23.2690 - mae: 23.2690
Epoch 2/5
1/1 [==============================] - 0s 10ms/step - loss: 22.0490 - mae: 22.0490
Epoch 3/5
1/1 [==============================] - 0s 9ms/step - loss: 20.8290 - mae: 20.8290
Epoch 4/5
1/1 [==============================] - 0s 17ms/step - loss: 19.6090 - mae: 19.6090
Epoch 5/5
1/1 [==============================] - 0s 11ms/step - loss: 18.3890 - mae: 18.3890


tf.int32

In [86]:
#trying make prediction based on X for y
model.predict([25])     #if our model is good enough, the answer would be around 35

1/1 [==============================] - 0s 50ms/step


array([[8.643126]], dtype=float32)

## Improving our model

### Creating new model to improve our result

In [87]:
from sklearn.model_selection import train_test_split
import numpy as np
X = np.array(X)
y = np.array(y)
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size = 0.2,random_state = True)

In [88]:
#create a model(specified to your probelm)
improved_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(100,activation = 'relu'),
    tf.keras.layers.Dense(1)
])

#compile model
improved_model.compile(loss = tf.keras.losses.mae,
                       optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                       metrics = ['mae'])

#fit the model with the subset of X and y and 100 epochs
improved_model.fit(tf.expand_dims(X_train,axis= 1),y_train,epochs = 100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 23.7662 - mae: 23.7662
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 23.4822 - mae: 23.4822
Epoch 3/100
1/1 [==============================] - 0s 12ms/step - loss: 23.1962 - mae: 23.1962
Epoch 4/100
1/1 [==============================] - 0s 10ms/step - loss: 22.9113 - mae: 22.9113
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 22.6325 - mae: 22.6325
Epoch 6/100
1/1 [==============================] - 0s 10ms/step - loss: 22.3642 - mae: 22.3642
Epoch 7/100
1/1 [==============================] - 0s 10ms/step - loss: 22.0940 - mae: 22.0940
Epoch 8/100
1/1 [==============================] - 0s 10ms/step - loss: 21.8321 - mae: 21.8321
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 21.5730 - mae: 21.5730
Epoch 10/100
1/1 [==============================] - 0s 10ms/step - loss: 21.3297 - mae: 21.3297
Epoch 11/100
1/1 [==============================] -

In [89]:
#let's see the result by doing prediction in the same way as we did before on 'model'
improved_model.predict([21])     #so much closer as you see, better and more usefull

1/1 [==============================] - 0s 104ms/step


array([[32.815945]], dtype=float32)